# Stage 6 Agent: Chat Interface

A clean, production-ready chat interface for the Stage 6 Full Memory agent.

**Features:**
- Working memory for conversation continuity
- Long-term memory with `search_memories()` and `store_memory()` tools
- Session management (New Session, Switch User)
- Expandable reasoning traces
- Performance metrics

---


## Part 1: Setup


In [1]:
# Set OpenAI API Key (REQUIRED)
# Option 1: Set it directly here (uncomment and add your key)
# import os
# os.environ["OPENAI_API_KEY"] = "sk-proj-your-key-here"

# Option 2: It will use the key from your shell environment if already set

import os
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY is not set!")
    print("   Please set it by uncommenting the lines above or running:")
    print("   export OPENAI_API_KEY='your-key-here'")
else:
    print(f"✅ OpenAI API Key is set ({len(os.environ['OPENAI_API_KEY'])} characters)")


✅ OpenAI API Key is set (164 characters)


In [2]:
# Enable nested event loops for Jupyter
import nest_asyncio
nest_asyncio.apply()

# Setup paths
import sys
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    project_root = cwd.parent
elif cwd.name == "ws" or (cwd / "progressive_agents").exists():
    project_root = cwd
elif (cwd / "materials" / "ws").exists():
    project_root = cwd / "materials" / "ws"
else:
    project_root = cwd

stage6_path = project_root / "progressive_agents" / "stage6_full_memory"
src_path = project_root / "src"
sys.path.insert(0, str(stage6_path))
sys.path.insert(0, str(src_path))

print(f"✅ Paths configured | Stage 6: {'Found' if stage6_path.exists() else 'Missing'}")


✅ Paths configured | Stage 6: Found


## Part 2: Initialize Agent


In [3]:
from agent import setup_agent
from agent.workflow import create_workflow, run_agent
from datetime import datetime

print("Initializing Stage 6 Full Memory Agent...")

# Setup agent
course_manager, memory_client = await setup_agent(auto_load_courses=True)

# Create workflow
workflow = create_workflow(course_manager, verbose=True)

# Initialize session
session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
student_id = "student_001"

print(f"""
✅ Agent Ready!
{'=' * 40}
Session: {session_id}
Student: {student_id}
""")


2026-02-21 14:12:13,477 - course-qa-setup - INFO - ================================================================================
2026-02-21 14:12:13,478 - course-qa-setup - INFO - Setting up Memory-Augmented Course Q&A Agent
2026-02-21 14:12:13,478 - course-qa-setup - INFO - ================================================================================
2026-02-21 14:12:13,478 - course-qa-setup - INFO - Initializing CourseManager with Redis URL: redis://localhost:6379
2026-02-21 14:12:13,478 - course-qa-setup - INFO - 📇 Using index: hierarchical_courses
2026-02-21 14:12:13,484 - redisvl.index.index - INFO - Index already exists, not overwriting.


Initializing Stage 6 Full Memory Agent...


2026-02-21 14:12:16,295 - course-qa-setup - INFO - 📚 Found 50 existing courses in Redis
2026-02-21 14:12:16,296 - course-qa-setup - INFO - ✅ CourseManager initialized with 50 courses
2026-02-21 14:12:16,297 - course-qa-setup - INFO - Initializing Agent Memory Server client: http://localhost:8088
2026-02-21 14:12:16,307 - course-qa-setup - INFO - ✅ Agent Memory Server client initialized
2026-02-21 14:12:16,307 - course-qa-setup - INFO - ================================================================================
2026-02-21 14:12:16,307 - course-qa-setup - INFO - ✅ Memory-Augmented Course Q&A Agent setup complete
2026-02-21 14:12:16,308 - course-qa-setup - INFO - ================================================================================
2026-02-21 14:12:16,317 - course-qa-workflow - INFO - Loaded 52 hierarchical courses for progressive disclosure



✅ Agent Ready!
Session: session_20260221_141216
Student: student_001



## Part 3: Chat Interface

Type your message at the top, chat history appears below.


In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import html as html_module
from datetime import datetime

# Global state
current_session_id = session_id
current_student_id = student_id
chat_messages = []

# Helper: Render reasoning trace
def render_reasoning_trace(trace):
    if not trace:
        return ""
    trace_html = '<div class="reasoning-trace"><details><summary>🧠 Show Reasoning Trace</summary><div class="reasoning-content">'
    for step in trace:
        step_type = step.get('type', 'unknown')
        content = html_module.escape(str(step.get('content', '')))
        trace_html += f'<div class="reasoning-step"><div class="step-type">{step_type.upper()}</div><div class="step-content">{content}</div></div>'
    trace_html += '</div></details></div>'
    return trace_html

# CSS styles for chat
CHAT_STYLES = """
<style>
    .user-message { background: #007bff; color: white; padding: 10px 14px; border-radius: 18px; margin: 4px 0; max-width: 65%; word-wrap: break-word; display: inline-block; }
    .agent-message { background: #f1f3f4; color: #202124; padding: 10px 14px; border-radius: 18px; margin: 4px 0; max-width: 80%; word-wrap: break-word; display: inline-block; }
    .message-wrapper { display: flex; margin-bottom: 16px; }
    .message-wrapper.user { justify-content: flex-end; }
    .message-wrapper.agent { justify-content: flex-start; }
    .metrics { font-size: 0.75em; color: #5f6368; margin-top: 8px; }
    .reasoning-trace { margin-top: 10px; border-top: 1px solid #dadce0; padding-top: 8px; }
    .reasoning-trace details { cursor: pointer; }
    .reasoning-trace summary { color: #1a73e8; font-weight: 500; font-size: 0.85em; }
    .reasoning-content { background: #f8f9fa; padding: 12px; margin-top: 8px; border-radius: 8px; border-left: 3px solid #1a73e8; font-size: 0.85em; }
    .reasoning-step { margin: 8px 0; padding: 6px 0; border-bottom: 1px solid #e8eaed; }
    .step-type { font-weight: 600; margin-bottom: 4px; }
    .step-content { margin-left: 20px; color: #3c4043; }
</style>
"""

# Helper: Update chat display
def update_chat_display():
    global chat_display_output
    with chat_display_output:
        clear_output()
        display(HTML(CHAT_STYLES))
        if not chat_messages:
            display(HTML('<div style="color: #5f6368; text-align: center; padding: 20px;">No messages yet. Type above to start!</div>'))
        else:
            for msg in chat_messages:
                if msg['role'] == 'user':
                    display(HTML(f'<div class="message-wrapper user"><div class="user-message">{html_module.escape(msg["content"])}</div></div>'))
                else:
                    content = html_module.escape(msg['content'])
                    metrics = msg.get('metrics', {})
                    iterations = msg.get('react_iterations', 0)
                    trace = msg.get('reasoning_trace', [])
                    metrics_html = f'<div class="metrics">⚡ {metrics.get("total_latency_ms", 0):.0f}ms | 🔄 {iterations} iterations</div>'
                    trace_html = render_reasoning_trace(trace)
                    display(HTML(f'<div class="message-wrapper agent"><div class="agent-message">{content}{metrics_html}{trace_html}</div></div>'))

# Session info widget
session_info = widgets.HTML(value=f'<div style="background: #e8f0fe; padding: 8px 12px; border-radius: 4px; font-size: 0.85em; margin-bottom: 12px; border-left: 3px solid #1a73e8;"><b>👤 Student:</b> {current_student_id} | <b>🔗 Session:</b> {current_session_id[:20]}...</div>')

# Buttons and input widgets
new_session_btn = widgets.Button(description='New Session', button_style='info', icon='plus', layout=widgets.Layout(width='130px', height='32px', margin='0 8px 0 0'))
new_user_btn = widgets.Button(description='Switch User', button_style='info', icon='user', layout=widgets.Layout(width='130px', height='32px'))
query_input = widgets.Textarea(placeholder='Type your message here...', layout=widgets.Layout(width='100%', height='80px', margin='0 0 8px 0'))
send_button = widgets.Button(description='Send', button_style='primary', icon='paper-plane', layout=widgets.Layout(width='110px', height='36px', margin='0 8px 0 0'))
clear_button = widgets.Button(description='Clear Chat', button_style='', icon='trash', layout=widgets.Layout(width='110px', height='36px'))
status_label = widgets.HTML(value='<span style="color: #5f6368;">💡 Ready to chat!</span>')

def send_message(b):
    global current_session_id, current_student_id, chat_messages
    query = query_input.value.strip()
    if not query:
        return
    query_input.disabled = True
    send_button.disabled = True
    status_label.value = '<span style="color: #1a73e8;">🤔 Agent is thinking...</span>'
    try:
        chat_messages.append({'role': 'user', 'content': query})
        query_input.value = ''
        update_chat_display()
        result = run_agent(workflow, query=query, session_id=current_session_id, student_id=current_student_id, enable_caching=False)
        chat_messages.append({'role': 'agent', 'content': result['final_response'], 'metrics': result.get('metrics', {}), 'react_iterations': result.get('react_iterations', 0), 'reasoning_trace': result.get('reasoning_trace', [])})
        update_chat_display()
        status_label.value = f'<span style="color: #0f9d58;">✅ Response received ({len(chat_messages)//2} messages)</span>'
    except Exception as e:
        status_label.value = f'<span style="color: #d93025;">❌ Error: {str(e)}</span>'
        import traceback
        traceback.print_exc()
    finally:
        query_input.disabled = False
        send_button.disabled = False

def new_session(b):
    global current_session_id, chat_messages
    current_session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    chat_messages.clear()
    update_chat_display()
    session_info.value = f'<div style="background: #e8f0fe; padding: 8px 12px; border-radius: 4px; font-size: 0.85em; margin-bottom: 12px; border-left: 3px solid #1a73e8;"><b>👤 Student:</b> {current_student_id} | <b>🔗 Session:</b> {current_session_id[:20]}...</div>'
    status_label.value = '<span style="color: #1a73e8;">🔄 New session started!</span>'

def new_user(b):
    global current_session_id, current_student_id, chat_messages
    user_num = int(datetime.now().strftime('%H%M%S')) % 1000
    current_student_id = f"student_{user_num:03d}"
    current_session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    chat_messages.clear()
    update_chat_display()
    session_info.value = f'<div style="background: #e8f0fe; padding: 8px 12px; border-radius: 4px; font-size: 0.85em; margin-bottom: 12px; border-left: 3px solid #1a73e8;"><b>👤 Student:</b> {current_student_id} | <b>🔗 Session:</b> {current_session_id[:20]}...</div>'
    status_label.value = '<span style="color: #1a73e8;">👤 Switched to new user!</span>'

def clear_chat(b):
    chat_messages.clear()
    update_chat_display()
    status_label.value = '<span style="color: #5f6368;">🗑️ Chat cleared</span>'

send_button.on_click(send_message)
clear_button.on_click(clear_chat)
new_session_btn.on_click(new_session)
new_user_btn.on_click(new_user)

# Layout - INPUT AT TOP
session_controls = widgets.HBox([new_session_btn, new_user_btn], layout=widgets.Layout(margin='0 0 12px 0'))
button_row = widgets.HBox([send_button, clear_button], layout=widgets.Layout(margin='0 0 8px 0'))
input_container = widgets.VBox([session_controls, query_input, button_row, status_label], layout=widgets.Layout(border='2px solid #1a73e8', border_radius='8px', padding='16px', background='#f8f9fa'))

# Chat display - BELOW INPUT (500px scrollable)
chat_display_output = widgets.Output(layout=widgets.Layout(height='500px', border='2px solid #1a73e8', border_radius='8px', padding='16px', background='white', overflow_y='auto', margin='16px 0 0 0'))

# Initial render and display
update_chat_display()
display(widgets.HTML('<h3 style="margin: 0 0 12px 0;">💬 Chat with Stage 6 Agent</h3>'))
display(session_info)
display(input_container)
display(widgets.HTML('<h4 style="margin: 20px 0 8px 0;">📜 Chat History</h4>'))
display(chat_display_output)

print("""✅ Chat interface ready!

Tips:
- Type your message at the top and click "Send"
- Chat history appears below and scrolls as it grows
- Use "New Session" for fresh conversation (same user)
- Use "Switch User" to test different user memories
- Click "🧠 Show Reasoning Trace" to see agent's thinking
""")


HTML(value='<h3 style="margin: 0 0 12px 0;">💬 Chat with Stage 6 Agent</h3>')

HTML(value='<div style="background: #e8f0fe; padding: 8px 12px; border-radius: 4px; font-size: 0.85em; margin-…

HTML(value='<h4 style="margin: 20px 0 8px 0;">📜 Chat History</h4>')

Output(layout=Layout(border_bottom='2px solid #1a73e8', border_left='2px solid #1a73e8', border_right='2px sol…

✅ Chat interface ready!

Tips:
- Type your message at the top and click "Send"
- Chat history appears below and scrolls as it grows
- Use "New Session" for fresh conversation (same user)
- Use "Switch User" to test different user memories
- Click "🧠 Show Reasoning Trace" to see agent's thinking

